# Import libraries

In [2]:
import torch
import json
from torch import nn
from tool import utils, preprocess, architecture, analysis
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import time
import os

# Before the analysis ...

## Get user input

In [3]:
ANALYSIS_NAME = 'Model1_Preproc' # Specify the folder name here (ex ANALYSIS_NAME = 'Model1_Preproc') ...
ANALYSIS_DIR_PATH = os.path.join('./RUN', ANALYSIS_NAME)
params = analysis.get_params(ANALYSIS_DIR_PATH)
params

{'ARCHITECTURE': '1', 'TREATMENT': 'yes'}

## Build test data loader

In [4]:
test_loader, img_size = analysis.get_test_loader(params)

Compose(
    CenterCrop(size=[300, 300])
    Resize(size=[224, 224], interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


## Build and load CNN model

In [5]:
model = analysis.build_load_model(params, ANALYSIS_DIR_PATH, img_size)

In [6]:
model

CNN_Model1(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=50176, out_features=18, bias=True)
)

# Analyses starts here ...

## Make predictions 

In [7]:
since = time.time()

true_label, predict = analysis.predict(model, test_loader)

time_elapsed = time.time() - since
print('Predictions on {} test images takes {:.0f}m {:.0f}s'.format(
    len(test_loader.dataset), time_elapsed // 60, time_elapsed % 60))

Predictions on 852 test images takes 0m 14s


## Accuracy

In [8]:
np.mean(true_label == predict) # 0.87

0.8532863849765259

## Confusion matrix

In [9]:
con = confusion_matrix(true_label, predict)
con_df = pd.DataFrame(con)
con_df.columns = test_loader.dataset.classes
con_df.index = test_loader.dataset.classes

In [10]:
con_df

,L_1,L_2,L_3,L_4,L_5,L_6,L_7,L_8,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,S_L,S_R
L_1,13,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
L_2,3,56,7,2,0,0,0,0,0,0,0,0,1,0,0,0,1,0
L_3,0,1,42,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
L_4,0,6,4,70,7,3,0,3,1,0,0,0,0,0,0,0,0,2
L_5,0,0,0,0,42,2,0,0,0,0,0,0,0,0,0,0,0,0
L_6,0,0,0,1,3,25,1,2,0,0,0,0,0,0,0,0,0,0
L_7,0,0,0,0,0,3,29,3,0,0,0,0,0,0,0,0,0,0
L_8,0,0,0,0,0,0,2,55,0,0,0,0,0,0,0,0,0,0
R_1,0,0,0,0,0,0,0,0,9,2,0,0,0,1,0,0,0,2
R_2,1,0,0,0,0,0,0,0,2,38,4,6,0,0,0,0,0,0


## Training logs

In [11]:
history = pd.read_csv(os.path.join(ANALYSIS_DIR_PATH, 'training_logger.csv'))

In [12]:
history

,Unnamed: 0,train_loss,train_acc,val_loss,val_acc
0,0,1.600377,0.490706,1.257199,0.603147
1,1,0.960796,0.707413,1.007631,0.707168
2,2,0.697147,0.783731,0.832988,0.753497
3,3,0.526630,0.838181,0.731951,0.812063
4,4,0.408627,0.869888,0.668027,0.829545
5,5,0.324595,0.892849,0.632117,0.846154
6,6,0.271656,0.909906,0.606482,0.860140
7,7,0.167664,0.951236,0.584516,0.868007
8,8,0.137998,0.960639,0.582824,0.870629
9,9,0.123723,0.966761,0.591112,0.870629
